In [4]:
# tools
import numpy as np

# data
from mlforecast.utils import generate_daily_series

# forecast
from mlforecast import MLForecast

# models
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

# data

In [23]:
series = generate_daily_series(n_series = 5)
print(series.head())
print()
print(series.shape)
print()
print(series["unique_id"].value_counts())

  unique_id         ds         y
0      id_0 2000-01-01  0.428973
1      id_0 2000-01-02  1.423626
2      id_0 2000-01-03  2.311782
3      id_0 2000-01-04  3.192191
4      id_0 2000-01-05  4.148767

(1101, 3)

unique_id
id_4    373
id_3    242
id_0    222
id_2    167
id_1     97
Name: count, dtype: int64


# forecast

In [24]:
fcst = MLForecast(
    models = {"lr": LinearRegression()},
    freq = "D",
    lags = [1],
    date_features = ["dayofweek"],
)

# training set

In [25]:
preps = fcst.preprocess(series)
print(preps.head())
print()
print(preps.shape)
print()
print(preps["unique_id"].value_counts())

  unique_id         ds         y      lag1  dayofweek
1      id_0 2000-01-02  1.423626  0.428973          6
2      id_0 2000-01-03  2.311782  1.423626          0
3      id_0 2000-01-04  3.192191  2.311782          1
4      id_0 2000-01-05  4.148767  3.192191          2
5      id_0 2000-01-06  5.028356  4.148767          3

(1096, 5)

unique_id
id_4    372
id_3    241
id_0    221
id_2    166
id_1     96
Name: count, dtype: int64


In [26]:
X = preps.drop(columns = ["unique_id", "ds", "y"])
y = preps["y"]
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X shape: (1096, 2)
y shape: (1096,)


# training

## training lr1

In [27]:
fcst.fit(series)

MLForecast(models=[lr], freq=D, lag_features=['lag1'], date_features=['dayofweek'], num_threads=1)

In [28]:
fcst.models_

{'lr': LinearRegression()}

## training lr2

In [18]:
fcst.fit_models(X, y)

MLForecast(models=[lr], freq=D, lag_features=['lag1'], date_features=['dayofweek'], num_threads=1)

In [19]:
fcst.models_

{'lr': LinearRegression()}

## training lgb

In [29]:
model = lgb.LGBMRegressor(n_estimators=100, verbosity = -1)
model.fit(
    X, 
    y, 
    eval_set = [(X, y)],
    categorical_feature = ["dayofweek"],
    callbacks = [lgb.log_evaluation(20)],
);

[20]	training's l2: 0.0823528
[40]	training's l2: 0.0230292
[60]	training's l2: 0.0207829
[80]	training's l2: 0.019675
[100]	training's l2: 0.018778


# computing forecasts

In [30]:
fcst.models_["lgbm"] = model
fcst.models_

{'lr': LinearRegression(), 'lgbm': LGBMRegressor(verbosity=-1)}

In [31]:
fcst.predict(h = 1)

,unique_id,ds,lr,lgbm
0,id_0,2000-08-10,3.549124,5.166797
1,id_1,2000-04-07,3.154285,4.252490
2,id_2,2000-06-16,2.880933,3.224506
3,id_3,2000-08-30,4.061801,0.245443
4,id_4,2001-01-08,2.904872,2.225106
